In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder 

In [2]:
CONFIG_PATH = "/home/astroturk/Desktop/Tensorflow/workspace/mobile_detection/exported-models/my_model/pipeline.config"
CHECKPOINT_PATH = "/home/astroturk/Desktop/Tensorflow/workspace/mobile_detection/exported-models/my_model/checkpoint/ckpt-0"
LABEL_PATH = "/home/astroturk/Desktop/Tensorflow/workspace/mobile_detection/annotations/label_map.pbtxt"

In [3]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(CHECKPOINT_PATH).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_PATH)

In [6]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [21]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

error: OpenCV(4.5.1) /tmp/pip-req-build-jhawztrk/opencv/modules/highgui/src/window.cpp:717: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'


In [14]:
ret, frame = cap.read(0)
print(frame.shape)
im2 = frame
im2[:, :, 0] = frame[:, :, 2]
im2[:, :, 2] = frame[:, :, 0]
plt.imshow(im2, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])
plt.savefig('ss.png')

(480, 640, 3)


In [7]:
print(width, height)

640 480


In [11]:
cap.release()